In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
  COLAB = True
except:
  # We are not in Google Colab
  COLAB = False
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# 4.0 Numerical Inverse Kinematics

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part I: Kinematics, Velocity, and Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[4.1 Numerical Inverse Kinematics Utility Class](#utility)
* Resolved-Rate Motion Control
  * Angle-axis error component
* Numerical Inverse Kinematics

[4.2 Newton-Raphson (NR) Method](#nr)
* Numerical Inverse Kinematics

[4.2 Newton-Raphson (NR) Method](#nr)
* Numerical Inverse Kinematics

[4.3 Gauss-Newton (GN) Method](#gn)
* Numerical Inverse Kinematics

[4.4 Levenberg-Marquardt (LM) Method](#lm)
* Numerical Inverse Kinematics

[4.5 Visualise IK Solutions](#vis)

[4.6 Numerical IK Comparison](#exp)

[4.7 Fast IK Solvers with the Robotics Toolbox](#rtb)

[4.8 Robotics Toolbox IK Speed Comparison](#speed)


In [1]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# ansitable is a great package for printing tables in a terminal
from ansitable import ANSITable

# python mechanisms to create abstract classes
from abc import ABC, abstractmethod

# a package for creating dynamic progress bars
from progress.bar import Bar

# swift is a lightweight browser-based simulator which comes with the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

# provides sleep functionaly
import time


<br>

<a id='utility'></a>

### 4.1 Numerical Inverse Kinematics Utility Class
---

Inverse kinematics deals is the problem of determining the corresponding joint coordinates, given some end-effector pose. Numerical inverse kinematics as opposed to analytical inverse kinematics uses an iterative technique.

We will start this Notebook by making an abstract base class called `IK`. This class implements everything needed to support a numerical inverse kinematics solver. It stores some general parameters typically used by any IK solver and also has some data structures useful for large-scale experiments. The idea behind the `IK` class is that a specific method class will inherit `IK` and implement the `step` method.

An important point to note is that numerical IK methods are subject to local minima and in such some will fail to converge on the solution. The choice of the initial joint configuration $\bf{q}_0$ is important.

An alternative approach is to re-start an IK problem with a new random $\bf{q}_0$ after a few $20 \sim 50$ iterations rather than persist with a single search attempt with $500 \sim 5000$ iterations. This is a simple but effective method of performing a global search for the IK solution. We can adjust this using the iteration limit `ilimit` and search limit `slimit` kwargs on the `IK` class.

In the Tutorial, we use a quadratic error term to describe the error between the current end-effector pose and desired end-effector pose. This error is consistent across each method we discuss.

Each method seeks to minimise the error function

\begin{align*}
    E = \frac{1}{2} \bf{e}^{\top} \bf{W}_e \bf{e}
\end{align*}

where $\bf{e} \in \mathbb{R}^6$ is the position and angle-axis error vector (see Notebook 3), and $\bf{W}_e = \text{diag}(\bf{w_e})(\bf{w_e} \in \mathbb{R}^n_{\geq0})$ is a diagonal weighting matrix which prioritises the corresponding error term. The `IK` class below takes a `we` vector in the `init` method and calculates the error vector `e` and quadratic error term `E` through the `error` method.

In [2]:
class IK(ABC):
    """
    An abstract super class which provides basic functionality to perform numerical inverse
    kinematics (IK). Superclasses can inherit this class and implement the solve method.

    This class also provides a mechanism to collect data on performance for large scale
    experiments.
    """

    def __init__(
        self,
        name: str = "IK Solver",
        ilimit: int = 30,
        slimit: int = 100,
        tol: float = 1e-6,
        we: np.ndarray = np.ones(6),
        problems: int = 1000,
    ):
        """
        name: The name of the IK algorithm
        ilimit: How many iterations are allowed within a search before a new search is started
        slimit: How many searches are allowed before being deemed unsuccessful
        tol: Maximum allowed residual error E
        we: A 6 vector which assigns weights to Cartesian degrees-of-freedom
        problems: Total number of IK problems within the experiment
        """

        # Solver parameters
        self.name = name
        self.slimit = slimit
        self.ilimit = ilimit
        self.tol = tol
        self.We = np.diag(we)

        # Solver results
        self.success = np.zeros(problems)
        self.searches = np.zeros(problems)
        self.iterations = np.zeros(problems)

        # initialise with NaN
        self.searches[:] = np.nan
        self.iterations[:] = np.nan
        self.success[:] = np.nan

    def solve(self, ets: rtb.ETS, Tep: np.ndarray, q0: np.ndarray):
        """
        This method will attempt to solve the IK problem and obtain joint coordinates
        which result the the end-effector pose Tep.

        The method returns a tuple:
        q: The joint coordinates of the solution (ndarray). Note that these will not
            be valid if failed to find a solution
        success: True if a solution was found (boolean)
        iterations: The number of iterations it took to find the solution (int)
        searches: The number of searches it took to find the solution (int)
        residual: The residual error of the solution (float)
        """

        # Iteration count
        i = 0
        total_i = 0

        for search in range(self.slimit):
            q = q0[search].copy()
            
            while i <= self.ilimit:
                i += 1

                # Attempt a step
                # try:
                E, q = self.step(ets, Tep, q)
                # except np.linalg.LinAlgError:
                #     i = np.nan
                #     break

                # Check if we have arrived
                if E < self.tol:
                    return q, True, total_i + i, search + 1, E

            total_i += i
            i = 0

        # If we make it here, then we have failed
        return q, False, np.nan, np.nan, E

    def error(self, Te: np.ndarray, Tep: np.ndarray):
        """
        Calculates the engle axis error between current end-effector pose Te and
        the desired end-effector pose Tep. Also calulates the quadratic error E
        which is weighted by the diagonal matrix We.

        Returns a tuple:
        e: angle-axis error (ndarray in R^6)
        E: The quadratic error weighted by We
        """
        e = rtb.angle_axis(Te, Tep)
        E = 0.5 * e @ self.We @ e

        return e, E

    @abstractmethod
    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        """
        Superclasses will implement this method to perform a step of the implemented
        IK algorithm
        """
        pass


<br>

<a id='nr'></a>

### 4.2 Newton-Raphson (NR) Method
---

The Newton-Raphson (NR) method seeks to minimise the error $E$ by iterating upon the following

\begin{align*}
    \bf{q}_{k+1} = \bf{q}_k + {^0\bf{J}(\bf{q}_k)}^{-1} \bf{e}_k.
\end{align*}

When using the NR method, the initial joint coordinates $q_0$, should correspond to a non-singular manipulator pose, since it uses the manipulator Jacobian.
When the the problem is solvable, it converges very quickly.
However, this method frequently fails to converge on the goal.

Most linear algebra libraries (including the Python `numpy` library) implement the pseudoinverse using singular value decomposition, which is robust to singular matrices. Therefore, we can this solver more robust by using the pseudoinverse instead of the normal inverse. 

The above equation requires the Jacobian to be square and non-singular. For a redundant manipulator (>6 DoF) or for more robust results to singularities we can use the pseudoinverse. We can use the pseudoinverse by supplying `pinv=True` when initialising the `NR` class.

In [3]:
class NR(IK):
    def __init__(self, pinv=False, **kwargs):
        super().__init__(**kwargs)
        self.pinv = pinv

        self.name = f"NR (pinv={pinv})"

    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        J = ets.jacob0(q)

        if self.pinv:
            q += np.linalg.pinv(J) @ e
        else:
            q += np.linalg.inv(J) @ e

        return E, q

<br>

<a id='gn'></a>

### 4.3 Gauss-Newton (GN) Method
---

We can improve the solvability of the NR method by using the Gauss-Newton (GN) method

\begin{align*}
    \bf{q}_{k+1} &= \bf{q}_k +
    \left(
    {\bf{J}(\bf{q}_k)}^\top
    \bf{W}_e \
    {\bf{J}(\bf{q}_k)}
    \right)^{-1}
    \bf{g}_k \\
    \bf{g}_k &=
    {\bf{J}(\bf{q}_k)}^\top
    \bf{W}_e
    \bf{e}_k
\end{align*}

where $\bf{J} = {^0\bf{J}}$ is the base-frame manipulator Jacobian. If $\bf{J}(\bf{q}_k)$ is non-singular, and $\bf{W}_e = \bf{1}_n$, then the above provides the pseudoinverse solution. However, if $\bf{J}(\bf{q}_k)$ is singular, the above can not be computed and the GN solution is infeasible.

Most linear algebra libraries (including the Python `numpy` library) implement the pseudoinverse using singular value decomposition, which is robust to singular matrices. Therefore, we can this solver more robust by using the pseudoinverse instead of the normal inverse. We can use the pseudoinverse by supplying `pinv=True` when initialising the `GN` class.

In [4]:
class GN(IK):
    def __init__(self, pinv=False, **kwargs):
        super().__init__(**kwargs)
        self.pinv = pinv

        self.name = f"GN (pinv={pinv})"

    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        if self.pinv:
            q += np.linalg.pinv(J.T @ self.We @ J) @ g
        else:
            q += np.linalg.inv(J.T @ self.We @ J) @ g

        return E, q

<br>

<a id='lm'></a>

### 4.4 Levenberg-Marquardt (LM) Method
---

We can further improve the solvability though the Levenberg-Marquardt (LM) Method

\begin{align*}
    \bf{q}_{k+1} 
    &= 
    \bf{q}_k +
    \left(
        \bf{A}_k
    \right)^{-1}
    \bf{g}_k \\
    %
    \bf{A}_k
    &=
    {\bf{J}(\bf{q}_k)}^\top
    \bf{W}_e \
    {\bf{J}(\bf{q}_k)}
    +
    \bf{W}_n
\end{align*}

where $\bf{W}_n = \text{diag}(\bf{w_n})(\bf{w_n} \in \mathbb{R}^n_{>0})$ is a diagonal damping matrix. The damping matrix ensures that $\bf{A}_k$ is non-singular and positive definite. The performance of the LM method largely depends on the choice of $\bf{W}_n$.

#### Wampler's Method

Wampler proposed $\bf{w_n}$ to be a constant. We provide the variable `λ` as a kwarg to the `LM_Wampler` class to adjust the constant.

In [5]:
class LM_Wampler(IK):
    def __init__(self, λ=1.0, **kwargs):
        super().__init__(**kwargs)
        
        self.name = f"LM (Wampler λ={λ})"
        self.λ = λ

    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        Wn = self.λ * np.eye(ets.n)
        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        q += np.linalg.inv(J.T @ self.We @ J + Wn) @ g

        return E, q

#### Chan's Method

Chan proposed

\begin{align*}
    \bf{W}_n
    &=
    \lambda E_k \bf{1}_n
\end{align*}

where $\lambda$ is a constant which reportedly does not have much influence on performance. We provide the variable `λ` as a kwarg to the `LM_Chan` class to adjust the weighting term.

In [6]:
class LM_Chan(IK):
    def __init__(self, λ=1.0, **kwargs):
        super().__init__(**kwargs)
        
        self.name = f"LM (Chan λ={λ})"
        self.λ = λ

    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        Wn = self.λ * E * np.eye(ets.n)
        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        q += np.linalg.inv(J.T @ self.We @ J + Wn) @ g

        return E, q


#### Sugihara's Method

Sugihara proposed

\begin{align*}
    \bf{W}_n
    &=
    E_k \bf{1}_n + \text{diag}(\hat{\bf{w}}_n)
\end{align*}

where $\hat{\bf{w}}_n \in \mathbb{R}^n$, $\hat{w}_{n_i} = l^2 \sim 0.01 l^2$, and $l$ is the length of a typical link within the manipulator.
We provide the variable `λ` as a kwarg to the `LM_Sugihara` class to adjust the value of $w_n$.

In [7]:
class LM_Sugihara(IK):
    def __init__(self, name="LM (Sugihara)", λ=1.0, **kwargs):
        super().__init__(name, **kwargs)

        self.name = f"LM (Sugihara λ={λ})"
        self.λ = λ

    def step(self, ets: rtb.ETS, Tep: np.ndarray, q: np.ndarray):
        Te = ets.eval(q)
        e, E = self.error(Te, Tep)

        Wn = E * np.eye(ets.n) + self.λ * np.eye(ets.n)
        J = ets.jacob0(q)
        g = J.T @ self.We @ e

        q += np.linalg.inv(J.T @ self.We @ J + Wn) @ g

        return E, q

<br>

<a id='vis'></a>

### 4.5 Visualise IK Solutions
---

_Note: This **section** of the notebook must be run locally. Due to how Swift operates, this section will not run on Google Colab_

We are going to try out some of our methods and visualise the results in Swift

In [8]:
# Make a panda robot
panda = rtb.models.Panda()
ets = panda.ets()

# Our IK methods do not respect joint limits. Therefore, many solutions
# are likely to be in violation of the joint limits. Swift enforces joint
# limits when stepping or simulating the environment. To properly
# visualise our results, we must remove the joint limits of the robot.
for link in panda.links:
    if link.isjoint:
        # link.qlim = None
        link.qlim = [-np.inf, np.inf]
        # link.qlim = [np.pi, -np.pi]

# Make an axes to visualise our current end-effector pose
ee_axes = sg.Axes(0.1)

# Make an axes to visualise our desired end-effector pose
goal_axes = sg.Axes(0.1)

def make_ik_env():
    # Make the environment
    env = Swift()

    # Launch the simulator
    env.launch(realtime=True, browser="notebook")

    # Add our robot to the simulator envionment
    env.add(panda)

    # Add axes to the environment
    env.add(ee_axes)
    env.add(goal_axes)

    return env

We will establish some goal poses and a general visualise method.

In [9]:
# Number of problems
problems = 10

# Iteration and seach limits
ilimit = 30
slimit = 100

# random valid q values for q0
q0 = ets.random_q(slimit)

# random valid q values which will define Tep
q_Tep = ets.random_q(problems)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_Tep[i])

def visualise_ik(solver, env):
    """
    A method which will test the solver over each pose in Tep
    and visualise the results in Swift
    """

    for pose in Tep:

        # Solver for Tep
        q, success, iterations, searches, residual = solver.solve(ets, pose, q0)

        # q = np.unwrap(q, period=np.pi)

        # print the results
        print(f"Successful: {success}, iterations: {iterations}, searches: {searches}, residual: {residual}")

        ### visualise the results
        # set the pandas joint coordinates to the solution q
        panda.q = q

        # set the goal axes to Tep pose
        goal_axes.T = pose

        # set the end-effector axes to the end-effector pose at the solution q
        ee_axes.T = panda.fkine(q)

        # step the environment to view the changes
        env.step(0)

        # sleep for 5 seconds to view the results
        time.sleep(2)

       

Lets test out the NR method first. Since we are using the Panda robot which has 7 DoF and is redundant we must set `pinv=True`.

In [11]:
# if not COLAB:
# Make a new Swift environment
env = make_ik_env()

# Make our solver
solver = NR(pinv=True, ilimit=30, slimit=100, problems=problems)

# Visualise the solver
visualise_ik(solver, env)

Successful: True, iterations: 239, searches: 8, residual: 1.2517220409813324e-10
Successful: True, iterations: 7, searches: 1, residual: 5.679160947330863e-07
Successful: True, iterations: 46, searches: 2, residual: 8.9906423315942e-10
Successful: True, iterations: 14, searches: 1, residual: 1.9964533602749153e-10
Successful: True, iterations: 77, searches: 3, residual: 1.3943978505293685e-08
Successful: True, iterations: 11, searches: 1, residual: 1.0541206022649218e-11
Successful: True, iterations: 13, searches: 1, residual: 3.764048383905955e-07
Successful: True, iterations: 11, searches: 1, residual: 9.948055341265228e-10
Successful: True, iterations: 21, searches: 1, residual: 1.1473899264832539e-07
Successful: True, iterations: 42, searches: 2, residual: 2.0369047430416716e-09


Visualise the LM Chan method

In [12]:
# if not COLAB:
# Make a new Swift environment
env = make_ik_env()

# Make our solver
solver = LM_Chan(λ=0.1, ilimit=30, slimit=100, problems=problems)

# Visualise the solver
visualise_ik(solver, env)

Successful: True, iterations: 13, searches: 1, residual: 2.1312535384825319e-07
Successful: True, iterations: 41, searches: 2, residual: 1.725205131999891e-07
Successful: True, iterations: 9, searches: 1, residual: 3.6808866827312603e-08
Successful: True, iterations: 6, searches: 1, residual: 2.163308461603359e-07
Successful: True, iterations: 9, searches: 1, residual: 8.756756916427679e-07
Successful: True, iterations: 8, searches: 1, residual: 6.702550304976584e-10
Successful: True, iterations: 43, searches: 2, residual: 8.683299710592003e-10
Successful: True, iterations: 7, searches: 1, residual: 2.473200254719037e-10
Successful: True, iterations: 8, searches: 1, residual: 1.4849017235606374e-07
Successful: True, iterations: 10, searches: 1, residual: 1.8750632498927855e-09


Feel free to modify the above cells to try out any of the other methods. You may have noticed that many of the solutions violated the joint limits of the panda. In Part II of the Tutorial we will address this issue.

<br>

<a id='exp'></a>

### 4.6 Numerical IK Comparison
---

In this Section we will perform a large scale experiment to compare the performance of each IK method.

This Notebook is intended to be set up as an IK playground. Feel free to modify, extend, or create your own methods as shown above. Then, simply add your solver to the `solvers` list below and text how it compares to the existing methods. Also feel free to try out different robots!

As shown above, the `ETS` class has its own method to generate random valid joint coordinate vectors. However, to make the table in the paper reporoducible, we replicate the method here.

In [13]:
def random_q(ets: rtb.ETS, i: int = 1) -> np.ndarray:
    """
    Generate a random valid joint configuration

    :param i: number of configurations to generate

    Generates a random q vector within the joint limits defined by
    `ets.qlim`.
    """

    if i == 1:
        q = np.zeros(ets.n)

        for i in range(ets.n):
            q[i] = np.random.uniform(ets.qlim[0, i], ets.qlim[1, i])

    else:
        q = np.zeros((i, ets.n))

        for j in range(i):
            for i in range(ets.n):
                q[j, i] = np.random.uniform(ets.qlim[0, i], ets.qlim[1, i])

    return q

What is shown below will recreate the data used for Table 1 in the Paper. It will take a while to run. If you want results faster, reduce the number `problems`.

In [14]:
# Our robot and ETS
robot = rtb.models.UR5()
ets = robot.ets()

# Setting the seed to 0 will make it possible to regenrate the table
# in the paper (provided no ther values were changed).
np.random.seed(0)

### Experiment parameters
# Number of problems to solve
problems = 10000

# Maximum slimit, we will preallocate the random q0 values so each
# solver uses the same set of random q0 values
max_slimit = 100

# random valid q values which will define Tep
q_Tep = ets.random_q(problems)

# random valid q values for q0
q0 = ets.random_q(max_slimit)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_Tep[i])

solvers = [
    #### Local Search
    NR(ilimit=500, slimit=1, problems=problems),
    GN(ilimit=500, slimit=1, problems=problems),
    NR(pinv=True, ilimit=500, slimit=1, problems=problems),
    GN(pinv=True, ilimit=500, slimit=1, problems=problems),
    LM_Wampler(λ=1e-4, ilimit=500, slimit=1, problems=problems),
    LM_Wampler(λ=1e-6, ilimit=500, slimit=1, problems=problems),
    LM_Chan(λ=1.0, ilimit=500, slimit=1, problems=problems),
    LM_Chan(λ=0.1, ilimit=500, slimit=1, problems=problems),
    LM_Sugihara(λ=0.001, ilimit=500, slimit=1, problems=problems),
    LM_Sugihara(λ=0.0001, ilimit=500, slimit=1, problems=problems),
    #### Global Search
    NR(problems=problems),
    GN(problems=problems),
    NR(pinv=True, problems=problems),
    GN(pinv=True, problems=problems),
    LM_Wampler(λ=1e-4, problems=problems),
    LM_Wampler(λ=1e-6, problems=problems),
    LM_Chan(λ=1.0, problems=problems),
    LM_Chan(λ=0.1, problems=problems),
    LM_Sugihara(λ=0.001, problems=problems),
    LM_Sugihara(λ=0.0001, problems=problems),
]

with Bar(message="Prog", max=problems, check_tty=False, hide_cursor=False) as bar:
    for i, pose in enumerate(Tep):

        for solver in solvers:
            (
                _,
                solver.success[i],
                solver.iterations[i],
                solver.searches[i],
                _,
            ) = solver.solve(ets, pose, q0)

        bar.next()
    bar.finish()


# Make a table to visualise the results
print(f"\nNumerical Inverse Kinematics Methods Compared over {problems} problems\n")

table = ANSITable(
    "Method",
    "sLimit",
    "iLimit",
    "Mean Iterations",
    "Median Iterations",
    "Infeasible",
    "Infeasible %",
    "Mean Searches",
    "Max Searches",
    border="thin",
)

for solver in solvers:
    table.row(
        solver.name,
        solver.slimit,
        solver.ilimit,
        np.round(np.nanmean(solver.iterations), 2),
        np.nanmedian(solver.iterations),
        np.sum(np.isnan(solver.iterations)),
        np.round(np.sum(np.isnan(solver.iterations)) / problems * 100.0, 2),
        np.round(np.nanmean(solver.searches), 2),
        np.nanmax(solver.searches),
    )


Prog |##                              | 877/10000


KeyboardInterrupt: 

In [15]:
table.print()

┌───────────────────────┬────────┬────────┬─────────────────┬───────────────────┬────────────┬──────────────┬───────────────┬──────────────┐
│                Method │ sLimit │ iLimit │ Mean Iterations │ Median Iterations │ Infeasible │ Infeasible % │ Mean Searches │ Max Searches │
├───────────────────────┼────────┼────────┼─────────────────┼───────────────────┼────────────┼──────────────┼───────────────┼──────────────┤
│       NR (pinv=False) │      1 │    500 │           21.06 │              16.0 │       1102 │        11.02 │           1.0 │          1.0 │
│       GN (pinv=False) │      1 │    500 │           21.41 │              16.0 │       1100 │         11.0 │           1.0 │          1.0 │
│        NR (pinv=True) │      1 │    500 │           21.15 │              16.0 │       1110 │         11.1 │           1.0 │          1.0 │
│        GN (pinv=True) │      1 │    500 │           21.91 │              16.0 │       1073 │        10.73 │           1.0 │          1.0 │
│ LM (Wampler

<br>

<a id='rtb'></a>

### 4.7 Fast IK Solvers with the Robotics Toolbox
---

In this section we will briefly demonstrate the IK solvers implemented in the Robotics Toolbox. 

The Toolbox has implemented each of the above IK techniques using C extensions and wrapped by python methods. The results generated from these methods will not perfectly replicate the above results as

* We can not use the same random number generator or seed
* The C implementations use different matrix inverse and pseudo-inverse calculators

These methods have some advanced parameters which we have not covered yet but will cover in Part II of the tutorial. These will be ignored here.

We make a simplified wrapper class for this experiment as we only wish to collect the data.

In [15]:
class RTB_IK:
    def __init__(self, name, solve, problems=problems):

        # Solver attributes
        self.name = name
        self.solve = solve

        # Solver results
        self.success = np.zeros(problems)
        self.searches = np.zeros(problems)
        self.iterations = np.zeros(problems)

        # initialise with NaN
        self.success[:] = np.nan
        self.searches[:] = np.nan
        self.iterations[:] = np.nan


We create the experiment parameters here

In [16]:
# Number of problems to solve
problems = 10000

# Cartesion DoF priority matrix
we = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# random valid q values which will define Tep
q_rand = ets.random_q(problems)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_rand[i])

# Maximum iterations allowed in a search
ilimit = 30

# Maximum searches allowed per problem
slimit = 100

# Solution tolerance
tol = 1e-6

We create a list of solvers similar to Section 4.6 but use `lambda` funtions to point to the solver we wish to use. 

In [17]:
rtb_solvers = [
    RTB_IK(
        "Newton Raphson (pinv=False)",
        lambda Tep: ets.ik_NR(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            pinv=False,
            joint_limits=False
        ),
        problems=problems,
    ),
    RTB_IK(
        "Gauss Newton (pinv=False)",
        lambda Tep: ets.ik_GN(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            pinv=False,
            joint_limits=False
        ),
        problems=problems,
    ),
    RTB_IK(
        "Newton Raphson (pinv=True)",
        lambda Tep: ets.ik_NR(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            pinv=True,
            joint_limits=False
        ),
        problems=problems,
    ),
    RTB_IK(
        "Gauss Newton (pinv=True)",
        lambda Tep: ets.ik_GN(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            pinv=True,
            joint_limits=False
        ),
        problems=problems,
    ),
    RTB_IK(
        "LM Wampler 1e-4",
        lambda Tep: ets.ik_LM(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=1e-4,
            joint_limits=False,
            method="wampler"
        ),
        problems=problems,
    ),
    RTB_IK(
        "LM Wampler 1e-6",
        lambda Tep: ets.ik_LM(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=1e-6,
            joint_limits=False,
            method="wampler"
        ),
        problems=problems,
    ),
    RTB_IK(
        "LM Chan 1.0",
        lambda Tep: ets.ik_LM(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=1.0,
            joint_limits=False,
            method="chan"
        ),
        problems=problems,
    ),
    RTB_IK(
        "LM Chan 0.1",
        lambda Tep: ets.ik_LM(
            Tep,
            q0=None,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=0.1,
            joint_limits=False,
            method="chan"
        ),
        problems=problems,
    ),
    RTB_IK(
        "LM Sugihara 0.001",
        lambda Tep: ets.ik_LM(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=0.001,
            joint_limits=False,
            method="sugihara"
        ),
        problems=problems,
    ),
    RTB_IK(
        "LM Sugihara 0.0001",
        lambda Tep: ets.ik_LM(
            Tep,
            ilimit=ilimit,
            slimit=slimit,
            tol=tol,
            mask=we,
            k=0.0001,
            joint_limits=False,
            method="sugihara"
        ),
        problems=problems,
    ),
]


Now we can run the experiment. This will be much faster than the Python implementations shown above.

In [18]:
with Bar(message="Prog", max=problems, check_tty=False, hide_cursor=False) as bar:
    for i, pose in enumerate(Tep):

        for solver in rtb_solvers:
            (
                _,
                solver.success[i],
                solver.iterations[i],
                solver.searches[i],
                _,
            ) = solver.solve(pose)

        bar.next()
    bar.finish()


# Make a table to visualise the results
print(f"\nNumerical Inverse Kinematics Methods Compared over {problems} problems\n")

table = ANSITable(
    "Method",
    "sLimit",
    "iLimit",
    "Mean Iterations",
    "Median Iterations",
    "Infeasible",
    "Infeasible %",
    "Mean Searches",
    "Max Searches",
    border="thin",
)

for solver in rtb_solvers:
    # print(solver.iterations)
    table.row(
        solver.name,
        slimit,
        ilimit,
        np.round(np.nanmean(solver.iterations), 2),
        np.nanmedian(solver.iterations),
        np.sum(np.isnan(solver.iterations)),
        np.round(np.sum(np.isnan(solver.iterations)) / problems * 100.0, 2),
        np.round(np.nanmean(solver.searches), 2),
        np.nanmax(solver.searches),
    )

table.print()

Prog |################################| 10000/10000




Numerical Inverse Kinematics Methods Compared over 10000 problems

┌────────────────────────────┬────────┬────────┬─────────────────┬───────────────────┬────────────┬──────────────┬───────────────┬──────────────┐
│                     Method │ sLimit │ iLimit │ Mean Iterations │ Median Iterations │ Infeasible │ Infeasible % │ Mean Searches │ Max Searches │
├────────────────────────────┼────────┼────────┼─────────────────┼───────────────────┼────────────┼──────────────┼───────────────┼──────────────┤
│Newton Raphson (pinv=False) │    100 │     30 │           29.72 │              18.0 │          0 │          0.0 │          1.49 │         22.0 │
│  Gauss Newton (pinv=False) │    100 │     30 │           29.38 │              17.0 │          0 │          0.0 │          1.47 │         21.0 │
│ Newton Raphson (pinv=True) │    100 │     30 │           28.98 │              17.0 │          0 │          0.0 │          1.46 │         19.0 │
│   Gauss Newton (pinv=True) │    100 │     30 │        

<br>

<a id='speed'></a>

### 4.8 Robotics Toolbox IK Speed Comparison
---

In this last section we will compare the speed of some of the Robotics Toolbox IK implementations. 

In [19]:
# Number of problems to solve
problems = 10000

# Cartesion DoF priority matrix
we = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

# random valid q values which will define Tep
q_rand = ets.random_q(problems)

# Our desired end-effector poses
Tep = np.zeros((problems, 4, 4))

for i in range(problems):
    Tep[i] = ets.eval(q_rand[i])

# Maximum iterations allowed in a search
ilimit = 30

# Maximum searches allowed per problem
slimit = 100

# Solution tolerance
tol = 1e-6

For the solver list, we aren't bothering with a wrapper class since we don't care about the stats, only the speed.

In [20]:
speed_solvers = [
    lambda Tep: ets.ik_NR(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        pinv=False,
        joint_limits=False,
    ),
    lambda Tep: ets.ik_GN(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        pinv=False,
        joint_limits=False,
    ),
    lambda Tep: ets.ik_NR(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        pinv=True,
        joint_limits=False,
    ),
    lambda Tep: ets.ik_GN(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        pinv=True,
        joint_limits=False,
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=1e-4,
        joint_limits=False,
        method="wampler",
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=1e-6,
        joint_limits=False,
        method="wampler",
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=1.0,
        joint_limits=False,
        method="chan",
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        q0=None,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=0.1,
        joint_limits=False,
        method="chan",
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=0.001,
        joint_limits=False,
        method="sugihara",
    ),
    lambda Tep: ets.ik_LM(
        Tep,
        ilimit=ilimit,
        slimit=slimit,
        tol=tol,
        mask=we,
        k=0.0001,
        joint_limits=False,
        method="sugihara",
    ),
]

speed_names = [
    "Newton Raphson (pinv=False)",
    "Gauss Newton (pinv=False)",
    "Newton Raphson (pinv=True)",
    "Gauss Newton (pinv=True)",
    "LM Wampler 1e-4",
    "LM Wampler 1e-6",
    "LM Chan 1.0",
    "LM Chan 0.1",
    "LM Sugihara 0.001",
    "LM Sugihara 0.0001",
]

times = []

In [22]:
for name, solver in zip(speed_names, speed_solvers):
    print(f"Solving with {name}")

    start = time.time()

    for i in range(problems):
        solver(Tep[i])

    total_time = time.time() - start
    times.append(total_time)


print(f"\nNumerical Inverse Kinematics Methods Times Compared over {problems} problems\n")

table = ANSITable(
    "Method",
    "Total Time (s)",
    "Average Time per Solution (μs)",
    border="thin",
)

for name, t in zip(speed_names, times):
    table.row(
        name,
        np.round(t, 4),
        np.round((t / problems) * 1e6, 4),
    )

table.print()

Solving with Newton Raphson (pinv=False)
Solving with Gauss Newton (pinv=False)
Solving with Newton Raphson (pinv=True)
Solving with Gauss Newton (pinv=True)
Solving with LM Wampler 1e-4
Solving with LM Wampler 1e-6
Solving with LM Chan 1.0
Solving with LM Chan 0.1
Solving with LM Sugihara 0.001
Solving with LM Sugihara 0.0001

Numerical Inverse Kinematics Methods Times Compared over 10000 problems

┌────────────────────────────┬────────────────┬────────────────────────────────┐
│                     Method │ Total Time (s) │ Average Time per Solution (μs) │
├────────────────────────────┼────────────────┼────────────────────────────────┤
│Newton Raphson (pinv=False) │         0.4279 │                        42.7927 │
│  Gauss Newton (pinv=False) │          0.655 │                         65.499 │
│ Newton Raphson (pinv=True) │         1.4902 │                       149.0235 │
│   Gauss Newton (pinv=True) │         1.5178 │                       151.7847 │
│            LM Wampler 1e-4 │